In [1]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [2]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

# writer embedding
director_df의 결측행, 중복행을 해결하기 ---> director의 행길이 : 6807

### 결측행 추가
director_df에 없는 영화정보를 director를 'nm0000000'으로해서 뒤에 추가

In [4]:
writer_df.head(3)

,item,writer
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005


In [6]:
no_wri_item_list = list(set(main_df['item']) - set(writer_df['item']))

In [7]:
no_wri_item_df = pd.DataFrame([x for x in zip(no_wri_item_list, ['nm0000000']*len(no_wri_item_list))])

In [8]:
no_wri_item_df.columns=writer_df.columns # 컬럼명 동일하게

In [9]:
writer_df = pd.concat([writer_df, no_wri_item_df]) # 기존 director_df 뒤에 작가없는 영화 추가

In [10]:
writer_df

,item,writer
0,1237,nm0000005
1,5147,nm0000005
2,7327,nm0000005
3,2068,nm0000005
4,7396,nm0000005
...,...,...
1154,55284,nm0000000
1155,81910,nm0000000
1156,4091,nm0000000
1157,2046,nm0000000


In [13]:
merged_writer_df = main_df.merge(writer_df, how='left')

In [14]:
merged_writer_df

,user,item,time,writer
0,11,4643,1230782529,nm0099541
1,11,4643,1230782529,nm0115310
2,11,4643,1230782529,nm0465199
3,11,4643,1230782529,nm0742797
4,11,170,1230782534,nm0000000
...,...,...,...,...
11591602,138493,68319,1260209720,nm1125275
11591603,138493,40819,1260209726,nm0003506
11591604,138493,40819,1260209726,nm0219456
11591605,138493,27311,1260209807,nm0004170


In [15]:
merged_writer_df.isna().sum()

user      0
item      0
time      0
writer    0
dtype: int64

### 아이템 별 작가정보 통계량으로 나타내기
한 아이템당 여러 작가가 참여했을 때, 작가들의 평균(interaction/movie_cnt) 평점중 가장 큰 수

In [19]:
# 아래코드 이해용
item_list = writer_df[writer_df['writer'] == 'nm0000005']['item'].tolist()

In [20]:
# 아래코드 이해용
item_list

[1237,
 5147,
 7327,
 2068,
 7396,
 4422,
 7939,
 7938,
 7941,
 7937,
 2131,
 34155,
 7936,
 7940,
 7577,
 7942,
 8405,
 7328]

In [21]:
# 아래코드 이해용
len(merged_writer_df[merged_writer_df['item'].isin(item_list)])

6020

In [381]:
# #작가별 받은 총 interaction 수 정보
# director_list = dir_df['director'].unique()
# director_dict = {}
# for name in director_list:
#     dir_item_list = director_df[director_df['director'] == name]['item'].tolist()   #해당 작가가 기여한 영화 리스트
#     director_dict[name] = len(dir_df[dir_df['item'].isin(dir_item_list)])           #해당 작가가 기여한 영화들의 총 interaction 수

In [22]:
#작가별 받은 총 interaction 수 정보 / 작가가 기여한 영화수
writer_list = merged_writer_df['writer'].unique()
writer_dict = {}
for name in writer_list:
    wri_item_list = writer_df[writer_df['writer'] == name]['item'].tolist()   #해당 작가가 기여한 영화 리스트
    writer_dict[name] = len(merged_writer_df[merged_writer_df['item'].isin(wri_item_list)]) / (len(wri_item_list)+1)      #해당 작가가 기여한 영화들의 총 interaction 수

In [23]:
writer_dict

{'nm0099541': 3076.8,
 'nm0115310': 4146.333333333333,
 'nm0465199': 1764.1,
 'nm0742797': 1764.1,
 'nm0000000': 269.56206896551726,
 'nm0003031': 5684.0,
 'nm0122364': 1005.0,
 'nm0027011': 6646.0,
 'nm0166307': 4469.4,
 'nm0166551': 4113.2,
 'nm0314788': 5445.076923076923,
 'nm0841035': 4465.6,
 'nm0858826': 5361.5,
 'nm0942723': 5794.0,
 'nm0001345': 2081.0,
 'nm0643973': 1036.3333333333333,
 'nm0694526': 4391.666666666667,
 'nm0694638': 4391.666666666667,
 'nm0000186': 2345.8,
 'nm0075015': 2182.0,
 'nm0212246': 1913.25,
 'nm0001279': 1925.0,
 'nm0078203': 413.6666666666667,
 'nm0932078': 616.3333333333334,
 'nm0549256': 967.0,
 'nm0568416': 159.25,
 'nm0000343': 1097.909090909091,
 'nm0083348': 9723.333333333334,
 'nm0308376': 1043.75,
 'nm0534681': 1574.7777777777778,
 'nm0730422': 3172.8571428571427,
 'nm0934093': 1574.7777777777778,
 'nm0000636': 1335.888888888889,
 'nm0527581': 2354.714285714286,
 'nm0809006': 2354.714285714286,
 'nm0000386': 4530.571428571428,
 'nm0460057': 7

In [25]:
writer_df['writer_inter'] = writer_df['writer'].map(writer_dict)

In [26]:
writer_df

,item,writer,writer_inter
0,1237,nm0000005,316.842105
1,5147,nm0000005,316.842105
2,7327,nm0000005,316.842105
3,2068,nm0000005,316.842105
4,7396,nm0000005,316.842105
...,...,...,...
1154,55284,nm0000000,269.562069
1155,81910,nm0000000,269.562069
1156,4091,nm0000000,269.562069
1157,2046,nm0000000,269.562069


In [27]:
# 결과 확인용
writer_df[writer_df['item']==1]

,item,writer,writer_inter
1942,1,nm0004056,28383.800000
2053,1,nm0005124,23612.714286
3512,1,nm0169505,15000.500000
3953,1,nm0230032,33020.428571
8254,1,nm0710020,42629.777778
9236,1,nm0812513,15000.500000
10205,1,nm0923736,10687.909091


In [28]:
df = writer_df.groupby('item')['writer_inter'].max()

In [29]:
df

item
1         42629.777778
2          5837.125000
3           510.625000
4          1336.307692
5          2538.818182
              ...     
118700      269.562069
118900     5791.285714
118997       96.000000
119141     1974.125000
119145     7050.285714
Name: writer_inter, Length: 6807, dtype: float64

In [ ]:
# 결과 확인용
writer_df[writer_df['item']==25764]

,item,writer,writer_inter
2841,25764,nm0090213,3067.000
3099,25764,nm0115669,653.875
4238,25764,nm0267868,245.750
6655,25764,nm0513858,296.000
7422,25764,nm0604526,230.000


In [31]:
writer_df = writer_df.merge(df, how='left', on='item')
writer_df

,item,writer,writer_inter_x,writer_inter_y
0,1237,nm0000005,316.842105,316.842105
1,5147,nm0000005,316.842105,316.842105
2,7327,nm0000005,316.842105,316.842105
3,2068,nm0000005,316.842105,316.842105
4,7396,nm0000005,316.842105,316.842105
...,...,...,...,...
12460,55284,nm0000000,269.562069,269.562069
12461,81910,nm0000000,269.562069,269.562069
12462,4091,nm0000000,269.562069,269.562069
12463,2046,nm0000000,269.562069,269.562069


In [32]:
writer_df['writer_info'] = writer_df['writer_inter_y']

In [33]:
writer_df.drop_duplicates(
        subset=["item"], keep="first", inplace=True)

In [34]:
writer_df

,item,writer,writer_inter_x,writer_inter_y,writer_info
0,1237,nm0000005,316.842105,316.842105,316.842105
1,5147,nm0000005,316.842105,316.842105,316.842105
2,7327,nm0000005,316.842105,316.842105,316.842105
3,2068,nm0000005,316.842105,316.842105,316.842105
4,7396,nm0000005,316.842105,316.842105,316.842105
...,...,...,...,...,...
12460,55284,nm0000000,269.562069,269.562069,269.562069
12461,81910,nm0000000,269.562069,269.562069,269.562069
12462,4091,nm0000000,269.562069,269.562069,269.562069
12463,2046,nm0000000,269.562069,269.562069,269.562069


In [37]:
writer_df[['item','writer_info']].to_csv(
    "writer_inter_cnt.csv", index=False
)

In [36]:
writer_df[['item','writer_info']]

,item,writer_info
0,1237,316.842105
1,5147,316.842105
2,7327,316.842105
3,2068,316.842105
4,7396,316.842105
...,...,...
12460,55284,269.562069
12461,81910,269.562069
12462,4091,269.562069
12463,2046,269.562069
